In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg


#Diagonally Implicit methods:
BE = rk.loadRKM('BE').__num__()
SDIRK23 = rk.loadRKM('SDIRK23').__num__()
SDIRK34 = rk.loadRKM('SDIRK34').__num__()
SDIRK54 = rk.loadRKM('SDIRK54').__num__()
TR_BDF2 = rk.loadRKM('TR-BDF2').__num__()


be = rk.loadRKM('BE').__num__()

#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()


from OrderCondition import *
from RKimple import *

Extrapolation Methods are row of implicit Euler Steps -> granted positifity?


# Define solver for EQ-system here to make it interchangeable

This is used to run implicit Euler Methods of the form

\begin{array}
{c|cccc}
c_1 & a_{11}\\
c_2 & a_{21} & a_{22} \\
\vdots & \vdots & & \ddots \\
c_s& a_{s1}& \cdots& \cdots &a_{ss}\\
\hline
& b_1 &b_2 &\cdots &b_s 
\end{array}


This leads to equation systems like (for the $i$th stage)

$k_i = f(t_n + c_n \Delta t,u+\Delta t (a_{i1}k_1+ \cdots +a_{ii}k_i))$

This can be writen as

$k_i = f(\underbrace{t_n + c_n \Delta t}_{t'},\underbrace{u+\Delta t (a_{i1}k_1+ \cdots +a_{ii-1}k_{i-1})}_{u'} + \Delta t a_{ii}k_i)$


A solver should solve the equation system

$x = f(t',u'+\Delta t a x)$ with $a = a_{ii}$

## Solver for linear Equations

$f(t,u) = A u$

$x=f(t',u'+ \Delta t a x) = A (u' + \Delta t a x)$

$-Au' = (\Delta t a A -I)x$

In [ ]:
# In RKimple.py
#def solver_Matrix(t,u,dt,a,A):
""" 
The function solves a equation system of the Form 
x = f(t,u+dt*a*x)
and returns x
where f(t,u)=Au
"""  
    

## Solver for nonlinear Equations with Fsolve

$x=f(t',u'+ \Delta t a x)$

$x-f(t',u'+ \Delta t a x) = 0$

or with $x = f(y)$

$y = u' + \Delta t a f(t,y)$

In [ ]:
# In RKimple.py

#def solver_nonlinear(t,u,dt,a,f):
""" 
The function solves a equation system of the Form 
x = f(t,u+dt*a*x)
and returns x
    
f is a function of t and u
""" 

#def solver_nonlinear_arg(t,u,dt,a,f):
""" 
The function solves a equation system of the Form 
x = f(t,u+dt*a*x)
and returns x
    
f is a function of t and u
""" 


# Decay

According to Bolley & Crouyeix every method with $\gamma = \inf$ has order $p\leq1$

Let's see if we can make a higher order Method positiv

In [ ]:
A = np.array([[-1]])


u0 =np.array([1.])

t_ref = np.linspace(0,100,1000)
u_ref = np.exp(-t_ref)*1

#Higher Order Method
t,u,b = RK_variable_b_implicit(ex3,10,A,w0=u0,t_final=100,b_fixed=True,solver_eqs = solver_Matrix)

t_pos,u_pos,b_pos = RK_variable_b_implicit(ex3,10,A,w0=u0,t_final=100,b_fixed=False,solver_eqs = solver_Matrix)

plt.plot(t_ref,u_ref)
plt.plot(t,u[0,:])
plt.plot(t_pos,u_pos[0,:])
plt.grid()

plt.xlim([5,15]);plt.ylim([-0.015,0.015])
#plt.xlim([5,15]);plt.ylim([-0.00015,0.00015])


#plt.xlim([15,25])
#plt.ylim([-0.00015,0.00015])

In [ ]:
print(u_pos)

Note: In this setup it makes sense to not directl project the negative $u$ on (or near) $u=0$ because the real solution is resasonable higher (e.g. with dt=10; plt.xlim([5,15]);plt.ylim([-0.00015,0.00015]))

In [ ]:
#Plot to show the deviation of the new b's (dotts) from the original b's (lines)
plt.plot(t_pos[1:],b_pos.T[1:,:],'o');
plt.plot(t[1:],b.T[1:,:]);

## Hamonic oscillator

In [ ]:

A = np.array([[0,-1],[1,0]])


u0 =np.array([1.,0.])

t,u,b = RK_variable_b_implicit(ex3,0.2,A,w0=u0,t_final=200,b_fixed=True,solver_eqs = solver_Matrix)




In [ ]:
plt.plot(t,u[0,:])

In [ ]:
def f_osc(t,u):
    return A@u

f_osc(0,u0)

t, u, b = RK_variable_b_implicit(ex3,0.2,f_osc,w0=u0,t_final=200,b_fixed=True,solver_eqs = solver_nonlinear_arg)

plt.plot(t,u[0,:])

In [ ]:
# Atmospheric pollution model from Hundsdorfer & Verwer

def k1(t):
    th = t/3600  # Time in hours
    thbar = th - 24*(th//24)
    if thbar>4 and thbar<20:
        sec = (np.sin(np.pi/16 *(thbar-4)))**0.2
        return 1.e-5*np.exp(7.*sec)
    else:
        return 1.e-40

k3 = 1.e-16
k2 = 1.e5
sigma2 = 1.e6

def f_pollution(t,u):
    du = np.zeros(4)
    du[0] = k1(t)*u[2] - k2*u[0]
    du[1] = k1(t)*u[2] - k3*u[1]*u[3] + sigma2
    du[2] = k3*u[1]*u[3]-k1(t)*u[2]
    du[3] = k2*u[0] - k3*u[1]*u[3]
    return du

In [ ]:
t_final = 3600*24*6
dt = 100
u0 = np.array([0.,1.3e8,5.e11,8.e11])


t, u, b = RK_variable_b_implicit(ex3,dt,f_pollution,w0=u0,t_final=t_final,solver_eqs =solver_nonlinear,
                                 b_fixed=True,solver=cp.SCS)



In [ ]:
plt.plot(np.array(t)/3600/24,u[0,:],'-k');
plt.xlim(0,6);
plt.grid()

In [ ]:
t_final = 3600*24*6
dt = 3600
u0 = np.array([0.,1.3e8,5.e11,8.e11])


t, u, b = RK_variable_b_implicit(ex8,dt,f_pollution,w0=u0,t_final=t_final,solver_eqs =solver_nonlinear_arg,
                                 b_fixed=True,solver=cp.SCS)




Apparently solving for the argument is adventagious

In [ ]:
plt.plot(np.array(t)/3600/24,u[0,:],'-k');
plt.xlim(0,6);
plt.grid()

In [ ]:
print(np.sum(u<-1e-12))
u[u<0]

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
def f_robertson(t,u):
    a = 0.3
    du = np.zeros(3)
    du[0] = 1e4 *u[1]*u[2] - 0.04*u[0]
    du[1] = 0.04 *u[0] - 1e4*u[1]*u[2] - 3e7*u[1]**2
    du[2] = 3e7*u[1]**2
    return du

u0 = np.array([1.,0.,0.])


t_stop = 10 # Time is 10^t_stop
steps = 10
dt = np.logspace(-5,t_stop,num=steps)[1:]-np.logspace(-5,t_stop,num=steps)[0:-1]

t, u, b = RK_variable_b_implicit(ex8,dt,f_robertson,w0=u0,t_final=10e3,solver_eqs =solver_nonlinear_arg,
                                 b_fixed=True,solver=cp.SCS)

In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:]*1e4)
plt.plot(t,u[2,:])
plt.xscale('log')

In [ ]:
print(np.sum(u<-1e-12))
u[u<0]

## Heat Equation

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]


u0 = (x>0.5)+0.
#dt = 0.7*dx**2
dt = 0.05

A_heat = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))


#t, u, b = RK_variable_b_implicit(ex8,dt,f_heat,w0=u0,t_final=1.1,solver_eqs =solver_nonlinear_arg,
#                                 b_fixed=True,solver=cp.SCS)

t, u, b = RK_variable_b_implicit(ex4,dt,A_heat,w0=u0,t_final=0.5,solver_eqs =solver_Matrix,
                                 b_fixed=False,solver=cp.SCS)



In [ ]:
plt.pcolor(u)
plt.colorbar()

In [ ]:
print(np.sum(u<-1e-12))
u[u<0]

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]
u0 = np.zeros_like(x)
u0[int(N/2)] = 1
#dt = 0.7*dx**2
dt = 0.0007847599703514606

A_heat = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))


t, u, b = RK_variable_b_implicit(ex3,dt,A_heat,w0=u0,t_final=0.01,solver_eqs =solver_Matrix,
                                 b_fixed=False,solver=cp.SCS,fallback = True)




In [ ]:
plt.pcolor(u)
plt.colorbar()

In [ ]:
plt.plot(u[:,1])
#plt.ylim(-1.1,1.1)
plt.grid()

In [ ]:
print(np.sum(u<-1e-12))
u[u<0]

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
plt.pcolor(u<0)

## Advection

The advection equation is spatially discretized using the upwind discretization. 

This can be represented using a Matrix $L$ of the structure

$$ L = \frac{1}{\Delta x}
\begin{pmatrix}
  -1 &  & &  \\
  1 & -1 & &  \\
   &  \ddots & \ddots &   \\
   &  & 1 & -1 
 \end{pmatrix}
$$

if we intodruce periodic boundary conditions this changes to 

$$ L_p = \frac{1}{\Delta x}
\begin{pmatrix}
  -1 &  & &  1\\
  1 & -1 & &  \\
   &  \ddots & \ddots &   \\
   &  & 1 & -1 
 \end{pmatrix}
$$

The boundary condition changes if there is a feasible $b$ to enshure positifity. 
In general it si easier to find a fesible $b$ for $L$. 
For further deatils see the Notebook on the Propagation Matrix

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]


A_upwind_advection = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))
#A_upwind_advection[0,-1] = 1/dx 




#u0 = (x<0.5)*1.
#u0 = (x<0.5)+0.2
#step
#u0 = (x<0.5)+0.
#u0 = np.sin(2*np.pi*x)+1.
#Delta function
u0 = np.zeros_like(x)
u0[int(N/2)] = 1
#u0[2] = 1

dt = 0.1

t, u, b = RK_variable_b_implicit(ex4,dt,A_upwind_advection,w0=u0,t_final=0.2,solver_eqs =solver_Matrix,
                                 b_fixed=False,solver=cp.SCS,fallback=True,num_fallback=1)




In [ ]:
plt.pcolor(u)
plt.colorbar()

In [ ]:
print(np.sum(u<-1e-12))
print(u[u<0])

In some cases the LP-solver returns a $b$ that does not comply with the positifity. Checking this in the plot.

In [ ]:
plt.pcolor(u<0)

In [ ]:
print(b)
b_store = b[:,1]

# Conclusion

It can be seen that integrating the ODEs usually doesn't lead to negative values. 
When integrating PDEs negative values occure. These can be seen at values where the solution is low innately.


Example: Heat Equation with $\Delta t = 0.001, t_{final}=0.01, u_0 = \delta $

Negative Values occur distant to the middle. These lead to a change of the b at the first step. 
By altering the b's the solution near the initial spike is wrong.

-> Some method to enshure that the b's have form that takes in account that some values are more important than others.

Open Question:
Using the Order Conditons of the original Order of the Problem?

Using the Order Condition of the original Order leads (at the Heat Equation) to larger b's. Question: Is it better to 

$\bullet$ reduce the Error by increasing the order of the Error Term and risking a greater magnitude at bigger stepsizes

$\bullet$ reduce the Error term by using b's that are close to the standard value, even if the method has not a high order.


The Advection Equation with periodic boundary conditions immeadetly gets infeasible in the tried examples and an high order. For some $u_0$ it works for the Advection equation without periodic boundary conditions. 
Another way to solve this is to reduce the Order to Order 1.


Note: are negative values mathematicly correct or are they artefacts from linalg solve beeing inaccurate? -> Tested, is correct. 

## Appendix: Exact solution with Matrix exponential


In [ ]:
from scipy.linalg import expm

def sol_matrix_exp(A,u0,dt,t_final):
    
    p = len(u0) #number of dimentions
    
    uu = np.zeros([p,int(t_final/dt)+1])
    uu[:,0] = u0.copy()
    tt = np.zeros([p,int(t_final/dt)+1])
    
    for i in range(1,int(t_final/dt)+1):
        uu[:,i] = expm(dt*i*A)@u0
        
    return (tt,uu)
     

In [ ]:
t, u = sol_matrix_exp(A_heat,u0,0.001,1)

plt.pcolor(u)
plt.colorbar()

# Appendix Convergence 

In [ ]:
def plot_convergence(time_integrator,rkm,f,u0,dt,refernce,step=1,error='abs',dx='1',Norm = 2,**Params):
    """"
    time_integrator: function used to integrate the ODE
    rkm,f,u0: Arguments for time integrator
    dt: dt array with dts
    reference: Array with reference solutions to compare the computet solution against 
    error: Definition of error computation, one of 'abs','rel','grid'
    dx: Discretisation for grid norm
    Norm: Norm to use for Error calculation ||u-u'||_Norm
    Params: Parameters for time integrator
    
    """
    
    err = np.zeros_like(dt)
    
    sol = []
    
    
    for i in range(dt.size):
        print('dt='+str(dt[i]))
        t,u,b = time_integrator(rkm,dt[i],f,**Params)
        dif = refernce[:,i]-u[:,step]
        if error == 'abs':
            err[i] = np.linalg.norm(dif,ord=Norm)
        elif error == 'rel':
            err[i] = np.linalg.norm(dif,ord=Norm)/np.linalg.norm(refernce[:,i],ord=Norm)
        elif error == 'grid': #Grid function Norm (LeVeque Appendix A.5)
            error[i] = dx**(1/Norm)*np.linalg.norm(dif,ord=Norm)
        else:
            print('Error not defined')
            print(error)
            raise InputError
        sol.append(u[:,step])
        
    plt.plot(dt,err,'o-')
    plt.yscale('log')
    plt.xscale('log')
    plt.grid()
    plt.ylabel('Error')
    plt.xlabel('dt')
    
    return sol,err
    
        

## Convergence for Advection

In [ ]:
from scipy.linalg import expm


N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]


A_upwind_advection = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))

u0 = np.zeros_like(x)
u0[int(N/3)] = 1



#dt = np.array([0.1,0.01,0.001,0.0001])
dt = np.logspace(-4,-0.5,num=10)

referencea_loc = np.zeros([len(u0),len(dt)])
for i in range(len(dt)):
    referencea_loc[:,i]= expm(dt[i]*A_upwind_advection)@u0



print('BE')
sol_a1,err_a1 = plot_convergence(RK_variable_b_implicit,be,A_upwind_advection,u0,dt,referencea_loc,error='rel',
                              w0=u0,t_final=0.3,b_fixed=True,solver=cp.SCS,fallback = True)

print('ex3 regular')
sol_a2,err_a2 = plot_convergence(RK_variable_b_implicit,ex3,A_upwind_advection,u0,dt,referencea_loc,error='rel',
                               w0=u0,t_final=0.3,b_fixed=True,solver=cp.SCS,fallback = False)

print('ex3 positiv')
sol_a3,err_a3 = plot_convergence(RK_variable_b_implicit,ex3,A_upwind_advection,u0,dt,referencea_loc,error='rel',
                               w0=u0,t_final=0.3,b_fixed=False,solver=cp.SCS,fallback = True,num_fallback = 2)


    
    

In [ ]:
#Plot global convergenc

referencea = np.zeros([len(u0),len(dt)])
for i in range(len(dt)):
    referencea[:,i]= expm(0.3*A_upwind_advection)@u0

print('BE')
sol_a4,err_a4 = plot_convergence(RK_variable_b_implicit,be,A_upwind_advection,u0,dt,referencea,error='rel',step = -1,
                              w0=u0,t_final=0.3,b_fixed=True,solver=cp.SCS,fallback = True)

print('ex3 regular')
sol_a5,err_a5 = plot_convergence(RK_variable_b_implicit,ex3,A_upwind_advection,u0,dt,referencea,error='rel',step = -1,
                               w0=u0,t_final=0.3,b_fixed=True,solver=cp.SCS,fallback = False)

print('ex3 positiv')
sol_a6,err_a6 = plot_convergence(RK_variable_b_implicit,ex3,A_upwind_advection,u0,dt,referencea,error='rel',step = -1,
                               w0=u0,t_final=0.3,b_fixed=False,solver=cp.SCS,fallback = True,num_fallback = 2)




In [ ]:
s = 8
plt.plot(sol_a4[s])
plt.plot(sol_a5[s])
plt.plot(sol_a6[s])
plt.plot(referencea[:,s])
plt.grid()

## Convergence for Heat Equation

In [ ]:
from scipy.linalg import expm


N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]
u0 = np.zeros_like(x)
u0[int(N/2)] = 1
#dt = 0.7*dx**2
dt = 0.001

A_heat = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))



#dt = np.array([0.1,0.01,0.001,0.0001])
dt = np.logspace(-5,-2,num=20)



reference_loc = np.zeros([len(u0),len(dt)])
for i in range(len(dt)):
    reference_loc[:,i]= expm(dt[i]*A_heat)@u0

print('BE')
sol_1,err_1 = plot_convergence(RK_variable_b_implicit,be,A_heat,u0,dt,reference_loc,error='rel',w0=u0,t_final=0.01,
                 b_fixed=True,solver=cp.SCS,fallback = False)

print('ex3 regular')
sol_2,err_2 = plot_convergence(RK_variable_b_implicit,ex3,A_heat,u0,dt,reference_loc,error='rel',w0=u0,t_final=0.01,
                 b_fixed=True,solver=cp.SCS,fallback = False)

print('ex3 positiv')
sol_3,err_3 = plot_convergence(RK_variable_b_implicit,ex3,A_heat,u0,dt,reference_loc,error='rel',w0=u0,t_final=0.01,
                 b_fixed=False,solver=cp.SCS,fallback = True)






#t, u, b = RK_variable_b_implicit(ex3,dt,A_heat,w0=u0,t_final=0.01,solver_eqs =solver_Matrix,
#                                 b_fixed=False,solver=cp.SCS,fallback = True)

In [ ]:
s = 12 #11 interesting
plt.plot(sol_1[s])
plt.plot(sol_2[s])
plt.plot(sol_3[s])
plt.plot(reference_loc[:,s]) 

In [ ]:
#Plot global convergenc

reference = np.zeros([len(u0),len(dt)])
for i in range(len(dt)):
    reference[:,i]= expm(0.01*A_heat)@u0

print('BE')
sol_4,err_4 = plot_convergence(RK_variable_b_implicit,be,A_heat,u0,dt,reference,error='rel',step = -1,
                              w0=u0,t_final=0.01,b_fixed=True,solver=cp.SCS,fallback = True)

print('ex3 regular')
sol_5,err_5 = plot_convergence(RK_variable_b_implicit,ex3,A_heat,u0,dt,reference,error='rel',step = -1,
                               w0=u0,t_final=0.01,b_fixed=True,solver=cp.SCS,fallback = False)

print('ex3 positiv')
sol_6,err_6 = plot_convergence(RK_variable_b_implicit,ex3,A_heat,u0,dt,reference,error='rel',step = -1,
                               w0=u0,t_final=0.01,b_fixed=False,solver=cp.SCS,fallback = True)

#plot_convergence(RK_variable_b_implicit,ex3,A_heat,u0,dt,reference,step = -1,w0=u0,t_final=0.01,
#                 b_fixed=False,solver=cp.SCS,fallback = True)

In [ ]:
s = 10
plt.plot(sol_4[s])
plt.plot(sol_5[s])
plt.plot(sol_6[s])
plt.plot(reference[:,0])
plt.grid()

#Check whether matrix exponential is correct...